<a href="https://colab.research.google.com/github/Aggarwal0741993/FINE_TUNING_IMBD_DATASET/blob/main/imbd_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets evaluate fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
from transformers import AutoTokenizer,TrainingArguments, Trainer,AutoModelForSequenceClassification,EarlyStoppingCallback
from datasets import load_dataset
import evaluate
import numpy as np

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [ ]:
import os
print(os.listdir('/kaggle/input/imdb-dataset-of-50k-movie-reviews'))

['IMDB Dataset.csv']


In [ ]:
from datasets import load_dataset, DatasetDict
import os

# Use the path obtained from kagglehub.dataset_download
csv_file_path = os.path.join(path, "IMDB Dataset.csv")

# Check if the file exists and has content before loading
if not os.path.exists(csv_file_path):
    raise FileNotFoundError(f"The file was not found at: {csv_file_path}")

# You might want to add a check for file size or read a small part to ensure it's not empty

try:
    dataset = load_dataset("csv", data_files=csv_file_path)
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")
    print("Please check the file content and format.")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 50000
    })
})

In [ ]:
from sklearn.model_selection import train_test_split
train_test=dataset["train"].train_test_split(test_size=0.2)
train_test

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 10000
    })
})

In [ ]:
test_dataset=train_test["test"]
train_dataset_temp=train_test["train"]

In [ ]:
train_valid=train_dataset_temp.train_test_split(test_size=0.4)
train_dataset=train_valid["train"]
valid_dataset=train_valid["test"]

In [ ]:
dataset=DatasetDict({"train":train_dataset,"test":test_dataset,"valid":valid_dataset})
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 10000
    })
    valid: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 16000
    })
})

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer=AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base",num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["review"], padding="max_length", truncation=True,max_length=128)
    result["labels"] = [1 if s == "positive" else 0 for s in examples["sentiment"]] # Convert sentiment to 0 or 1
    return result

tokenized_dataset=dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset=tokenized_dataset.remove_columns(["review", "sentiment"])

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16000
    })
})

In [ ]:
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16000
    })
})

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(eva_pred):
  logits, labels = eva_pred
  predict = np.argmax(logits, axis=1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, predict, average='binary')
  acc = accuracy_score(labels, predict)
  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [ ]:
#tokenized_dataset['train']=tokenized_dataset["train"].select(range(10000))
#tokenized_dataset['valid']=tokenized_dataset["test"].select(range(1000))

In [ ]:
from transformers import TrainingArguments

training_args=TrainingArguments(output_dir="./results",eval_strategy="epoch",save_strategy="epoch",learning_rate=2e-5,
                                per_device_train_batch_size=8,per_device_eval_batch_size=8,num_train_epochs=2,
                                weight_decay=0.03,logging_dir="./logs",logging_steps=50, report_to="none",
                                load_best_model_at_end=True,
                                greater_is_better=False,
                                metric_for_best_model="eval_loss")

In [ ]:
trainer=Trainer(model=model,args=training_args,train_dataset=tokenized_dataset["train"],
                eval_dataset=tokenized_dataset["valid"],compute_metrics=compute_metrics,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=3)])

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.304700,0.383921,0.897312,0.896491,0.909730,0.883631
2,0.245100,0.417274,0.903500,0.904609,0.900049,0.909215


TrainOutput(global_step=6000, training_loss=0.31468456888198854, metrics={'train_runtime': 1573.9203, 'train_samples_per_second': 30.497, 'train_steps_per_second': 3.812, 'total_flos': 3157332664320000.0, 'train_loss': 0.31468456888198854, 'epoch': 2.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_dataset["test"])

{'eval_loss': 0.35748112201690674,
 'eval_accuracy': 0.9017,
 'eval_f1': 0.8990033905270729,
 'eval_precision': 0.9133611691022965,
 'eval_recall': 0.8850900262998179,
 'eval_runtime': 75.7024,
 'eval_samples_per_second': 132.096,
 'eval_steps_per_second': 16.512,
 'epoch': 2.0}

In [ ]:
trainer.save_model("imbd_fine_tuned")
tokenizer.save_pretrained("imbd_fine_tuned")

('imbd_fine_tuned/tokenizer_config.json',
 'imbd_fine_tuned/special_tokens_map.json',
 'imbd_fine_tuned/vocab.json',
 'imbd_fine_tuned/merges.txt',
 'imbd_fine_tuned/added_tokens.json',
 'imbd_fine_tuned/tokenizer.json')

In [ ]:
import shutil
shutil.make_archive("imbd_fine_tuned", 'zip', "imbd_fine_tuned")

'/content/imbd_fine_tuned.zip'

In [ ]:
from google.colab import files
files.download('imbd_fine_tuned.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>